<a href="https://colab.research.google.com/github/xalejandrow/NLP-Project-Tutorial/blob/main/Clase_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
import numpy as np
import nltk
import re
import unicodedata
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/machine-learning-content/master/assets/spam.csv')

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [12]:
df_raw.sample(10)

,Category,Message
1946,ham,Can ü call me at 10:10 to make sure dat i've w...
2722,ham,I'm working technical support :)voice process.
3676,ham,Whos this am in class:-)
787,ham,It does it on its own. Most of the time it fix...
4281,spam,WINNER!! As a valued network customer you have...
1432,ham,"My painful personal thought- ""I always try to ..."
4812,ham,E admin building there? I might b slightly ear...
4667,ham,Not..tel software name..
1443,ham,"Its ok., i just askd did u knw tht no?"
4415,ham,How stupid to say that i challenge god.You don...


In [13]:
# Tratar de llevar la información de texto a minúsculas
df_raw['Message'] = df_raw['Message'].str.lower()

In [14]:
#df_raw.sample(10)

In [16]:
# Elementos de texto qeu no son informativos - stopwords
# Tratar de encontrar esas palabras

df_raw['Message'].str.split(expand=True).stack().value_counts()[:60]

to           2234
i            2217
you          1921
a            1433
the          1326
u             996
and           968
is            868
in            857
my            755
for           704
your          677
of            614
me            611
have          568
call          556
on            521
are           487
that          470
it            466
2             457
so            423
but           422
or            415
not           411
at            400
can           386
ur            385
if            382
with          379
will          379
i'm           377
be            376
get           375
do            364
just          363
we            346
this          309
when          283
from          277
&lt;#&gt;     276
go            265
up            264
all           261
no            258
4             255
how           254
what          252
now           247
.             241
like          236
got           235
know          230
was           230
free          228
out       

In [24]:
# Función para Retirar las stopword
stop = stopwords.words('english')
def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop:
        words_filtered.append(word)
    result = " ".join(words_filtered)
  else:
    result = None
  return result


In [26]:
df_interin = df_raw.copy()

In [27]:
df_interin['Message'] = df_interin['Message'].apply(remove_stopwords)

In [28]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u            996
call         556
2            457
ur           385
i'm          377
get          375
&lt;#&gt;    276
go           265
4            255
.            241
like         236
got          235
know         230
free         228
come         220
good         201
?            187
send         187
want         183
text         175
time         169
i'll         168
...          163
love         163
going        161
ok           160
ü            157
need         157
r            153
still        151
one          150
txt          149
n            146
see          145
dont         140
new          136
tell         135
think        127
reply        126
mobile       124
take         124
back         121
stop         119
please       118
home         112
day          111
&            111
hi           108
claim        104
hope         103
make         101
give         100
pls           99
me.           98
phone         97
now.          94
later         94
happy         93
much          

In [31]:
# Utilizaremos regex regular expression
df_interin['Message'] = df_interin['Message'].str.replace(".","", regex=False)

In [42]:
# Primer regex
df_interin['Message'] = df_interin['Message'].str.replace("""[?&#,;.ü']""","", regex=True)

In [43]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u         1105
call       573
2          472
im         461
ur         386
get        382
go         277
ltgt       276
ok         275
4          269
free       253
know       249
like       243
ill        238
got        237
good       232
come       228
time       205
want       193
send       190
love       189
text       186
day        183
going      170
one        170
me         167
need       166
lor        160
home       160
you        156
see        155
still      154
now        154
sorry      153
r          153
stop       150
back       150
txt        149
dont       147
n          146
reply      143
tell       137
new        136
later      134
think      132
mobile     128
today      128
hi         128
well       127
it         127
take       126
please     125
da         125
cant       124
phone      118
claim      111
night      111
much       109
dear       109
hey        107
dtype: int64

In [46]:
# Función adicional 
# Para las palabras en español

def normalize_string(message):
  if message is not None:
    result = unicodedata.normalize('NFC',message).encode('ascii','ignore').decode()
  else:
    result = None
  return result

In [47]:
df_interin['Message'] = df_interin['Message'].apply(normalize_string)

In [48]:
# Es común que escriban gooodd loovveee o etc etc.

def replace_multiple_letters(message):
  if message is not None:
    result = re.sub(r"([a-zA-Z])\1{2,}",r"\1",message)
  else:
    result = None
  return result

In [49]:
df_interin['Message'] = df_interin['Message'].apply(replace_multiple_letters)

In [50]:
df_interin['Message'].str.split(expand=True).stack().value_counts()[:60]

u         1105
call       573
2          472
im         470
ur         386
get        382
go         277
ltgt       276
ok         275
4          269
free       253
know       249
like       243
ill        242
got        237
good       232
come       228
time       205
want       193
love       190
send       190
text       186
day        183
one        170
going      170
me         167
need       166
home       160
lor        160
dont       158
you        157
see        155
now        154
still      154
sorry      153
r          153
stop       150
back       150
txt        149
n          146
reply      143
tell       137
new        136
later      134
think      132
today      128
well       128
hi         128
mobile     128
it         127
cant       126
take       126
please     126
da         125
phone      118
claim      111
night      111
oh         110
much       109
dear       109
dtype: int64

In [53]:
df = df_interin.copy()

In [55]:
X = df['Message']
y = df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=121)

In [56]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [64]:
X_train
#filas = 4179 mensajes
# columnas = 7559 palabras
# Esto es para el train - faltan los de test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [58]:
X_train.shape

(4179, 7559)

In [63]:
# Para ver los valores de la matríz
vec.get_feature_names_out()


array(['008704050406', '0089', '0121', ..., 'zoe', 'zogtorius', 'zoom'],
      dtype=object)

In [73]:
nb = MultinomialNB()

In [75]:
nb.fit(X_train, y_train)

MultinomialNB()

In [76]:
nb.score(X_train, y_train)

0.9952141660684374

In [77]:
nb.score(X_test, y_test)

0.9820531227566404

In [78]:
# No es spam
nb.predict(vec.transform(['hello my friend']))

array(['ham'], dtype='<U4')

In [79]:
# Es spam
nb.predict(vec.transform(['free for season']))

array(['spam'], dtype='<U4')